In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.AdvancedSolvers;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
////================================
//// pt2 of this sheet: Process Data
////================================
// Load Tables
//string basepath = @"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\strongScaling\IBM_RotCube\";
string basepath = System.IO.Directory.GetCurrentDirectory()+@"\";
List<System.Data.DataTable> LoadedTabs = new List<System.Data.DataTable>();
//LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"strong_scaling_3_2021-9-24.json"));
LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"weak_scaling_5_1_p-MG_2021-10-13.json"));
//LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"strong_scaling_3_1_2021-9-24.json"));
//LoadedTabs.Add(TableExtensions.LoadFromFile(basepath+"strong_scaling_3_3_2021-9-.json"));
var LoadTab = LoadedTabs.Pick(0);
LoadedTabs.ForEach(s=>LoadTab.Merge(s)); LoadTab.Rows.Count()

6

In [ ]:
string[] ColumnNames = new string[]{               
   "OrthonormalizationMultigrid.AddSol",
   "Schwarz.Solve",
   "OrthonormalizationMultigrid.MinimizeResidual",
   "OrthonormalizationMultigrid.Residual",
   "OrthonormalizationMultigrid.Solve",
   "overlap_scaling",
   "block_solve_level",
   "caching",
   "nocaching",
   "OrthonormalizationMultigrid.VerivyCurrentResidual",
   "Restriction",
   "Prolongation",
   "SolverRun_time",
   "LSolverIter",
   "DGdegree:Velocity*",
   "DOF",
   "MGDepth",
   "NoOfNodes",
   "NoOfCores",
   "DirectSolver.Solve",
};

In [ ]:
var SesTab = LoadTab.ExtractColumns(ColumnNames);

In [ ]:
// Average of Repititions ... pt1: Group Runs
var RegistryOfRuns = new Dictionary<string,List<System.Data.DataRow>>();
foreach(System.Data.DataRow Row in SesTab.Rows){
    string test = "";
    test += "k"+Convert.ToString(Row["DGdegree:Velocity*"]);
    test += "_c"+Convert.ToString(Row["NoOfCores"]);
    List<System.Data.DataRow> Reps;
    if(RegistryOfRuns.TryGetValue(test,out Reps)){
        Reps.Add(Row);
    } else {
        Reps = new List<System.Data.DataRow>();
        Reps.Add(Row);
        RegistryOfRuns.Add(test,Reps);
    }
} RegistryOfRuns.OrderBy(s=>s.Key[0]).OrderBy(s=>s.Key[1]).Select(s=>new object[]{s.Key,s.Value.Count()})

index,value
0,"[ k4_c256, 1 ]"
1,"[ k4_c128, 1 ]"
2,"[ k4_c64, 1 ]"
3,"[ k4_c32, 1 ]"
4,"[ k4_c16, 1 ]"
5,"[ k4_c8, 1 ]"


In [ ]:
// Average of Repititions ... pt2: Compute Average
var tmp = SesTab.Clone();
tmp.Clear();
foreach(var kvp in RegistryOfRuns){
    var newrow = tmp.NewRow();
    var sample = kvp.Value.First();
    newrow["DGdegree:Velocity*"]=Convert.ToInt32(sample["DGdegree:Velocity*"]);
    newrow["NoOfCores"]=Convert.ToInt32(sample["NoOfCores"]);
    foreach(System.Data.DataColumn col in tmp.Columns){
        if(col.ColumnName=="DGdegree:Velocity*"||col.ColumnName=="NoOfCores")
            continue;
        double value=0.0;
        foreach(System.Data.DataRow row in kvp.Value){
            value+=Convert.ToDouble(row[col.ColumnName]);
        }
        newrow[col]=value / kvp.Value.Count();
    }
    tmp.Rows.Add(newrow);
}
SesTab = tmp;
SesTab.Print()

In [ ]:
//string thisdir = Directory.GetCurrentDirectory();
//SesTab.ToCSVFile(thisdir+@"\table.csv");

In [ ]:
// Minitab blaupause 4 Runtime per iteration
var Minitab_blueprint = new System.Data.DataTable();
Minitab_blueprint.Columns.Add("NoOfCores", typeof(Int32));
Minitab_blueprint.Columns.Add("runtime_per_iter", typeof(Double));
Minitab_blueprint.Columns.Add("Speedup", typeof(Double));

In [ ]:
// fill minitables for plots
var listofDGdegree = new List<int>();
foreach(System.Data.DataRow Row in SesTab.Rows){
    listofDGdegree.Add(Convert.ToInt32(Row["DGdegree:Velocity*"]));
}
List<double> DGdegrees=listofDGdegree.GroupBy(s=>s).Select(s=>s.Average()).ToList();
string dirpath = Directory.GetCurrentDirectory();
foreach(double DG in DGdegrees){
    var MiniTab = Minitab_blueprint.Clone();
    foreach(System.Data.DataRow Row in SesTab.Rows){
        if(Convert.ToInt32(Row["DGdegree:Velocity*"])!=DG) continue;
        var newrow = MiniTab.NewRow();
        newrow["NoOfCores"] = Convert.ToInt32(Row["NoOfCores"]);
        double iter = Convert.ToDouble(Row["LSolverIter"]);
        double runtime = Convert.ToDouble(Row["SolverRun_time"]);
        newrow["runtime_per_iter"] = runtime / iter;
        MiniTab.Rows.Add(newrow);
    }
    System.Data.DataView view = new System.Data.DataView(MiniTab);
    view.Sort = "NoOfCores";
    MiniTab=view.ToTable();

    double reference = Convert.ToDouble(MiniTab.Rows[0]["runtime_per_iter"]);
    foreach(System.Data.DataRow Row in MiniTab.Rows){
        Row["Speedup"]=reference/Convert.ToDouble(Row["runtime_per_iter"]);
    }

    MiniTab.ToCSVFile(dirpath+@"\strong_k"+DG+".dat");
} dirpath

b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\strongScaling\IBM_RotSphere

In [ ]:
// Generate rnd Formatting for massive multiplots
var theDict = new List<Tuple<string,DashTypes,LineColors>>();
public static Tuple<string,DashTypes,LineColors>[] AofFormats;
var dashes = new DashTypes[]{DashTypes.Dashed,DashTypes.DotDashed,DashTypes.DotDotDashed};
var points= new PointTypes[]{PointTypes.OpenCircle,PointTypes.OpenCircle,PointTypes.OpenDiamond,PointTypes.OpenUpperTriangle};
var colors = new LineColors[]{LineColors.Red, LineColors.Green, LineColors.Blue, LineColors.Magenta, LineColors.Yellow, LineColors.Black, LineColors.Orange, LineColors.Grey};

int iSweep=0;
foreach(string Name in SesTab.GetColumnNames()){
    //if(Name!="OrthonormalizationMultigrid.MinimizeResidual"&&Name!="overlap_scaling")
    //    continue;
    if(Name=="NoOfCores"||Name=="LSolverIter"||Name=="DGdegree:Velocity*"||Name=="NoOfNodes"||Name=="MGDepth"||Name=="DOF")
        continue;
    int one=iSweep%(dashes.Length-1);
    int two=iSweep/(dashes.Length-1);
    theDict.Add(new Tuple<string,DashTypes,LineColors>(Name,dashes[one],colors[two]));
    iSweep++;
}
AofFormats = theDict.ToArray()

index,Item1,Item2,Item3
0,OrthonormalizationMultigrid.AddSol,Dashed,Red
1,Schwarz.Solve,DotDashed,Red
2,OrthonormalizationMultigrid.MinimizeResidual,Dashed,Green
3,OrthonormalizationMultigrid.Residual,DotDashed,Green
4,OrthonormalizationMultigrid.Solve,Dashed,Blue
5,overlap_scaling,DotDashed,Blue
6,block_solve_level,Dashed,Magenta
7,caching,DotDashed,Magenta
8,nocaching,Dashed,Yellow
9,OrthonormalizationMultigrid.VerivyCurrentResidual,DotDashed,Yellow


In [ ]:
public static class DoWhatUWant{
    public static PlotRowSelectorEx Solver_DG_Proj(int DG){
    return delegate (int iSweep, int iTabRow, 
                    IDictionary<string, object> Row, 
                    out string Nmn, 
                    out PlotFormat Fmt,
                    out double xValue, out double yValue) { 

            int k  = Convert.ToInt32(Row["DGdegree:Velocity*"]);
            Nmn = AofFormats[iSweep].Item1;

            if(k != DG ) {
                // degree does not match -> not in this plot
                Nmn    = null;
                Fmt    = null;
                xValue = 0;
                yValue = 0;
                return;
            } 
            
            Fmt           = new PlotFormat();
            Fmt.PointSize = 0.5;
            Fmt.Style     = Styles.LinesPoints;
            Fmt.LineWidth = 3;

            Fmt.DashType = AofFormats[iSweep].Item2;
            Fmt.LineColor = AofFormats[iSweep].Item3;
            double iter = Convert.ToDouble(Row["LSolverIter"]);
            yValue = Convert.ToDouble(Row[Nmn])/iter;
            //yValue = Convert.ToDouble(Row[Nmn]);
            xValue    = Convert.ToDouble(Row["NoOfCores"]);
        };
    }

    public static PlotRowSelectorEx Solver_DG_Proj(){
        return delegate (int iSweep, int iTabRow, 
                    IDictionary<string, object> Row, 
                    out string Nmn, 
                    out PlotFormat Fmt,
                    out double xValue, out double yValue) { 

            int k           = Convert.ToInt32(Row["DGdegree:Velocity*"]);
            
            Fmt           = new PlotFormat();
            Fmt.PointSize = 0.5;
            Fmt.Style     = Styles.LinesPoints;
            Fmt.LineWidth = 3;
            
            switch(k){
            case 2:
                Nmn="k2";
                Fmt.LineColor = LineColors.Red;
                break;
            case 3:
                Nmn="k3";
                Fmt.LineColor = LineColors.Orange;
                break;
            case 4:
                Nmn="k4";
                Fmt.LineColor = LineColors.Blue;
                break;
            default:
                throw new Exception("vergiss es");
            }
            double time = Convert.ToDouble(Row["SolverRun_time"]);
            double iter = Convert.ToDouble(Row["LSolverIter"]);
            double timeSchwarz = Convert.ToDouble(Row["Schwarz.Solve"]);
            //yValue = Convert.ToDouble(Row["NoOfNodes"]);
            yValue = time/iter;
            //yValue = Convert.ToDouble(Row["MGDepth"]);
            //yValue=Convert.ToDouble(Row["LSolverIter"]);
            //yValue = iter;
            //yValue = time - timeSchwarz;
            xValue = Convert.ToDouble(Row["NoOfCores"]);
        };
    }
}

In [ ]:
int k = 4;
int xMin = 8;
int xMax = 256;

In [ ]:
Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
//multiplots[0,0] = SesTab.ToPlot(DoWhatUWant.Solver_DG_Proj(),AofFormats.Length);
multiplots[0,0] = SesTab.ToPlot(DoWhatUWant.Solver_DG_Proj(k),AofFormats.Length);
multiplots[0,0].LogX = true;
multiplots[0,0].LogY = false;
multiplots[0,0].tmargin = 0;
multiplots[0,0].bmargin = 5;
multiplots[0,0].XrangeMin=xMin;
multiplots[0,0].XrangeMax=xMax;
multiplots[0,0].YrangeMin=0;
multiplots[0,0].YrangeMax=2;
multiplots[0,0].Ylabel = "iter";
multiplots[0,0].Xlabel = "cores";
multiplots[0,0].LegendAlignment=new string[]{"i","t","r"};
multiplots.PlotNow()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 OrthonormalizationMultigrid.AddSol 
 
 
 
 
 OrthonormalizationMultigrid.AddSol 
 
 
 
 
 
 
 
 
 
 
 
 
 Schwarz.Solve 
 
 
 Schwarz.Solve 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.MinimizeResidual 
 
 
 OrthonormalizationMultigrid.MinimizeResidual 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.Residual 
 
 
 OrthonormalizationMultigrid.Residual 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.Solve 
 
 
 OrthonormalizationMultigrid.Solve 
 
 
 
 
 
 
 
 
 
 
 
 
 overlapscaling 
 
 
 overlap s caling 
 
 
 
 
 
 
 
 
 
 
 
 
 blocksolvelevel 
 
 
 block s olve l evel 
 
 
 
 
 
 
 
 
 
 
 
 
 caching 
 
 
 caching 
 
 
 
 
 
 
 
 
 
 
 
 
 nocaching 
 
 
 nocaching 
 
 
 
 
 
 
 
 
 
 
 
 
 OrthonormalizationMultigrid.VerivyCurrentResidual 
 
 
 OrthonormalizationMultigrid.VerivyCurrentResidual 
 
 
 
 
 
 
 
 
 
 
 
 
 Restriction 
 
 
 Restriction 
 
 
 
 
 
 
 
 
 
 
 
 
 Prolongation 
 
 
 Prolongation 
 
 
 
 
 
 
 
 
 
 
 
 
 SolverRuntime 
 
 
 SolverRun t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 DirectSolver.Solve 
 
 
 DirectSolver.Solve